In [11]:
from op_op.load import load_events, load_motion, load_raw_calcium
from op_op.alignment import demarcate_trials, demarcate_latency_from_event
from op_op.event_feature_eng import (
    num_prev_trials,
    cumulative_reward,
    reward_in_window,
    cumulative_shock,
    shock_in_window,
    prev_iti_length,
    trial_start_delay,
    screen_touch_delay,
    reward_collection_delay,
    trial_duration,
)

import os
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display

In [2]:
DATA_DIR = Path(os.environ.get("DATA_DIR")) / "raw"
MOUSE_NAME = "example-mouse"
SESSION_NAME = "rdt-d1"

In [6]:
df_events = load_events(DATA_DIR, MOUSE_NAME, SESSION_NAME)
df_events.head(3)

,trial_idx,block_type,was_omission,was_forced_choice,start_time,mouse_init_time,screen_touch_time,reward_collection_time,chose_large,was_shocked
0,1,1,False,True,78.241,78.842,81.672,82.344,True,False
1,2,1,False,True,90.344,95.799,98.387,99.676,True,False
2,3,1,False,True,107.676,108.766,109.963,110.787,True,False


#### Add Event Features

In [9]:
df = df_events.copy()


df["num_prev_trials"] = num_prev_trials(df_events)
df["num_prev_trials_in_block"] = num_prev_trials(df_events, in_block=True)
df["cumulative_reward"] = cumulative_reward(df_events)
df["cumulative_shock"] = cumulative_shock(df_events)
df["reward_in_window"] = reward_in_window(
    df_events, window_kernel=np.ones(5)
)  # 5 trials
df["shock_in_window"] = shock_in_window(df_events, window_kernel=np.ones(5))  # 5 trials
df["prev_iti_length"] = prev_iti_length(df_events)
df["trial_start_delay"] = trial_start_delay(
    df_events, shift=0
)  # shift=0 for current trial, 1 for previous trial
df["screen_touch_delay"] = screen_touch_delay(df_events, shift=0)
df["reward_collection_delay"] = reward_collection_delay(df_events, shift=0)
df["trial_duration"] = trial_duration(df_events)

cols = [
    "trial_idx",
    "num_prev_trials",
    "num_prev_trials_in_block",
    "cumulative_reward",
    "cumulative_shock",
    "reward_in_window",
    "shock_in_window",
    "prev_iti_length",
    "trial_start_delay",
    "screen_touch_delay",
    "reward_collection_delay",
    "trial_duration",
]
display(df[cols].head(3))

,trial_idx,num_prev_trials,num_prev_trials_in_block,cumulative_reward,cumulative_shock,reward_in_window,shock_in_window,prev_iti_length,trial_start_delay,screen_touch_delay,reward_collection_delay,trial_duration
0,1,0,0,0.0,0.0,0.0,0.0,NaN,0.601,2.830,0.672,NaN
1,2,1,1,1.0,0.0,1.0,0.0,8.0,5.455,2.588,1.289,4.103
2,3,2,2,2.0,0.0,2.0,0.0,8.0,1.090,1.197,0.824,9.332
